### Code Hist.

 - CODE  
    &ensp; : Crawling - 특일 정보 조회 (KASI)

  - DATE  
    &ensp; 2023-11-29 Created  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1)   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2)    
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 3)   
    
 - DESC  
    &ensp; : 전처리 - 한국지역난방공사 열판매량/열공급량   
    &emsp; 1) 결측치가 없어서, 그대로 사용  
    &emsp;&ensp;&ensp; 
    &emsp;&ensp;&ensp; (Crawl Code 없음)   
    &emsp; 2) 

 - DATA  
    &emsp; <"Input">  
    1) None (Input Dataset)  
    &emsp;- Period :   
    &emsp;- Interval : 

    &emsp; <"Output">  
    1) Hourly (관측소/년도별 출력)  
    &nbsp;df_data_cal.to_csv(data_dir + 'KASI_DATE_D_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    2) Daily (관측소/년도별 출력)  
    &nbsp;df_data_cal_24.to_csv(data_dir + 'KASI_DATE_H_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['locdate', 'YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'HOUR', 'MINUTE']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    

 - Related Link  
    &ensp; : 

# 01. Code

## 01-01. Init

### 01-01-01. Init_Module Import

In [1]:
#region Basic_Import
## Basic
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import random
from pandas import DataFrame, Series

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## TSNE
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

## 정규화
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
## Modeling, Model Training
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
## Model 평가
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score   # model.score   

## Excel/CSV
import openpyxl, xlrd

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [2]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [3]:
## Import_Local (현재 프로그램에서 미사용)
from DEV_Common_Data_Analysis import print_desc_statistic
from DEV_Common_Data_Datetime import create_col_ymdhm, create_col_datetime, create_df_dt, conv_midnight_24to00, create_col_weekdays #, validate_date, list_invalidDate, calc_df_dt
from DEV_Common_Data_Preprocessing import resample_by_last, find_outlier_Usages, del_outlier_Usages
from DEV_Common_Data_Visualization import visualization_df
from DEV_KASI_Holiday import KASI_holiDay, KASI_restDay, KASI_anniDay
from DEV_KMA_Weather_ASOS import KMA_ASOS_DATA
from DEV_KDHC_Usage import KDHC_HEAT_Usage

### 01-01-02. Config (Directory, Params)

In [4]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [5]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y = pd.datetime.now().year
str_now_m = pd.datetime.now().month
str_now_d = pd.datetime.now().day
str_now_hr = pd.datetime.now().hour
str_now_min = pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2023-11-30 04:01:34.192647
2023 / 11 / 30
4 : 1


## 01-02. Data Load (df_raw)

### 01-02-01. KDHC Heat Usage

In [31]:
# Define data root directory
str_dir_kdhcHeat = "./data_Energy_KDHC_National_Heat/"
print(os.listdir(str_dir_kdhcHeat))

df_kdhc_heat = pd.read_csv(str_dir_kdhcHeat + 'KDHC_COMB_HEATxWEATHER_2019-2022.csv', index_col = 0)
df_kdhc_heat

['DATA_KDHC_Heat_Provide_Cheongju_2010_2021.csv', 'DATA_KDHC_Heat_Provide_Cheongju_2010_2021_1st_Check.csv', 'DATA_KDHC_Heat_Provide_Sejong_2014_2018.csv', 'DATA_KDHC_Heat_Provide_Sejong_2014_2018_1st_Check.csv', 'DATA_KDHC_Heat_Sell_Branch_201910_202109.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022_complete_analysis.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022_mean_imp.csv', 'KDHC_NATIONAL_HEAT_hr.csv', 'KDHC_NATIONAL_HEAT_RAW_hr.csv']


,METER_DATE,day_of_the_week,YEAR,MONTH,DAY,HOUR,MINUTE,code_day_of_the_week,rest_YN,dist_from_holiday,...,SUWON,HWASUNG,DONGTAN,PYONGTAEK,CHEONGJU,SEJONG,KIMHAE,DAEGU,YANGSAN,GWANGJU_JEONNAM
0,2019-10-01 01:00:00,Tuesday,2019,10,1,1,0,1,0,2,...,26.0000000000,11.0000000000,21,0.5000000000,16,18.0000000000,8,19.0000000000,10,3
1,2019-10-01 01:00:00,Tuesday,2019,10,1,1,0,1,0,2,...,26.0000000000,11.0000000000,21,0.5000000000,16,18.0000000000,8,19.0000000000,10,3
2,2019-10-01 02:00:00,Tuesday,2019,10,1,2,0,1,0,2,...,29.0000000000,12.0000000000,16,0.5000000000,13,15.0000000000,7,16.0000000000,9,2
3,2019-10-01 02:00:00,Tuesday,2019,10,1,2,0,1,0,2,...,29.0000000000,12.0000000000,16,0.5000000000,13,15.0000000000,7,16.0000000000,9,2
4,2019-10-01 03:00:00,Tuesday,2019,10,1,3,0,1,0,2,...,33.0000000000,12.0000000000,14,0.5000000000,10,15.0000000000,7,15.0000000000,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27298,2022-09-30 20:00:00,Friday,2022,9,30,20,0,4,0,1,...,73.0000000000,39.0000000000,61,12.0000000000,36,44.0000000000,12,30.0000000000,18,6
27299,2022-09-30 21:00:00,Friday,2022,9,30,21,0,4,0,1,...,71.0000000000,36.0000000000,55,4.0000000000,30,44.0000000000,11,35.0000000000,22,5
27300,2022-09-30 22:00:00,Friday,2022,9,30,22,0,4,0,1,...,65.0000000000,29.0000000000,45,2.0000000000,27,43.0000000000,12,32.0000000000,22,6
27301,2022-09-30 23:00:00,Friday,2022,9,30,23,0,4,0,1,...,64.0000000000,25.0000000000,32,3.0000000000,25,40.0000000000,11,34.0000000000,22,6


In [32]:
col_tar = 'PYONGTAEK'
list_outlierRow = find_outlier_Usages(df_kdhc_heat, col_tar)
list_outlierRow = list(reversed(list_outlierRow))
print(list_outlierRow)

print_desc_statistic(df_kdhc_heat, 'PYONGTAEK')
# list_col = ['PAJU', 'GOYANG', 'SAMSONG'
#             , 'JOONGANG'
#             , 'KANGNAM'
#             , 'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON', 'HWASUNG', 'DONGTAN'
#             , 'PYONGTAEK', 'CHEONGJU', 'SEJONG'
#             , 'KIMHAE', 'DAEGU', 'YANGSAN'
#             , 'GWANGJU_JEONNAM']

# for col in list_col:
#     visualization_df(df_kdhc_heat, col)

14.0
2.4
21
[22343, 22223, 22151, 22050, 22031, 21994, 21982, 21876, 21864, 21582, 21574, 21503, 21441, 21373, 21358, 21208, 21192, 21130, 21119, 20854, 19430]
=============== Descriptive Statistic ===============
0.0
5.374228697584508
4.6
6.2829982053254225
93.0
=============== Descriptive Statistic ===============


### 01-03. Create DF_Anomaly Delete

### 01-03-01. 완전분석법(Completes Analysis) : 모든 결측값을 삭제

In [33]:
df_complete_analysis = pd.DataFrame.copy(df_kdhc_heat)
print(df_complete_analysis.shape)
df_complete_analysis = df_complete_analysis.drop(index = list_outlierRow, axis = 0)
print(df_complete_analysis.shape)
df_complete_analysis.to_csv(str_dir_kdhcHeat + 'KDHC_COMB_HEATxWEATHER_2019-2022_complete_analysis.csv')

(27303, 46)
(27282, 46)


### 01-03-02. 평균 대치법(Mean Imputation) : 평균값으로 대체

In [34]:
df_mean_imp = pd.DataFrame.copy(df_kdhc_heat)
flt_mean = np.mean(df_mean_imp[col_tar])
print(flt_mean)
for i in list_outlierRow:
    print(df_mean_imp[col_tar].iloc[i])
    df_mean_imp[col_tar].iloc[i] = flt_mean
    print(df_mean_imp[col_tar].iloc[i])
df_mean_imp.to_csv(str_dir_kdhcHeat + 'KDHC_COMB_HEATxWEATHER_2019-2022_mean_imp.csv')

6.2829982053254225
38.0
6.2829982053254225
33.0
6.2829982053254225
36.0
6.2829982053254225
37.0
6.2829982053254225
33.0
6.2829982053254225
37.0
6.2829982053254225
43.0
6.2829982053254225
38.0
6.2829982053254225
39.0
6.2829982053254225
36.0
6.2829982053254225
37.0
6.2829982053254225
32.0
6.2829982053254225
32.0
6.2829982053254225
35.0
6.2829982053254225
36.0
6.2829982053254225
37.0
6.2829982053254225
36.0
6.2829982053254225
32.0
6.2829982053254225
37.0
6.2829982053254225
38.0
6.2829982053254225
93.0
6.2829982053254225


### 01-03-03. 1~2주전 동일 시간 대치

In [15]:
df_prevWeek_imp = pd.DataFrame.copy(df_kdhc_heat)
for i in list_outlierRow:
    print(df_mean_imp[col_tar].iloc[i])
    df_mean_imp[col_tar].iloc[i] = 
    print(df_mean_imp[col_tar].iloc[i])

### 01-03-04. IQR 탐지 기반 Lienar Regression

In [37]:
df_linearReg = pd.DataFrame.copy(df_kdhc_heat)

df_linearReg = del_outlier_Usages(df_linearReg, col_tar)
find_outlier_Usages(df_linearReg, col_tar)
df_linearReg.to_csv(str_dir_kdhcHeat + 'KDHC_COMB_HEATxWEATHER_2019-2022_linearReg.csv')

14.0
2.4
21
14.0
2.4
0


In [ ]:
## 회귀 대치법(Regression Imputation) : 

## 단순확률 대치법(Single Stochastic Imputation) : 

## 최근접 대치법 (Nearest-Neighbor Imputation) : 계층

## 


## 최근접+평균 대치법 : 계층을 나눈 후, 앞뒤값의 평균에 해당하는 계층의 평균값으로 대치

## 동일요일 / 시간대의 평균값으로 대치
df_sameWeekdayTime = pd.DataFrame.copy(df_kdhc_heat)